In [1]:
import json

with open("data/gamestate_output/SNGS-001.json", "r") as f:
    data = json.load(f)
    predictions = data["predictions"]

In [2]:
print(len(predictions))

12376


In [3]:
for prediction in predictions[:30]:
    print(json.dumps(prediction, indent=4))

{
    "image_id": "4001000001",
    "bbox_image": {
        "x": 382.20587158203125,
        "y": 783.1085815429688,
        "w": 71.101806640625,
        "h": 177.919189453125
    },
    "video_id": "001",
    "category_id": 1.0,
    "track_id": 1,
    "bbox_pitch": {
        "x_bottom_left": -6.3148218321410114,
        "y_bottom_left": 23.00920891811881,
        "x_bottom_right": -5.6064553448488725,
        "y_bottom_right": 23.014673568178488,
        "x_bottom_middle": -5.960646729080064,
        "y_bottom_middle": 23.011941180348604
    },
    "supercategory": "object",
    "attributes": {
        "role": "player",
        "jersey": "24",
        "team": "left"
    },
    "id": "0"
}
{
    "image_id": "4001000001",
    "bbox_image": {
        "x": 1064.484375,
        "y": 493.9342041015625,
        "w": 56.9696044921875,
        "h": 149.6256103515625
    },
    "video_id": "001",
    "category_id": 1.0,
    "track_id": 2,
    "bbox_pitch": {
        "x_bottom_left": 0.62503567

In [4]:
!pip install -e visualization/ 

Obtaining file:///Users/dariogod/coding/ugent/thesis_playground/invoke_gamestate/visualization
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for visualization (pyproject.toml) ... done
  Created wheel for visualization: filename=visualization-0.0.1-0.editable-py3-none-any.whl size=2771 sha256=93063c62472ea1adeb57168be046f7c21175edf9828147532895a6a6366707f5
  Stored in directory: /private/var/folders/6l/p6zt7z7j3qz18vr9n_6dpf3w0000gn/T/pip-ephem-wheel-cache-9_1x0x0i/wheels/04/f6/2c/685b13e65d0683e01515b854ce058514aef692573ea731d26c
Successfully built visualization
  Attempting uninstall: visualization
    Found existing installation: visualization 0.0.1
    Uninstalling visualization-0.0.1:
      Successfully uninstalled visualization-0.0.1


In [5]:
import os
from pathlib import Path
import shutil
from typing import Optional
from visualization.visualizer import draw_frame, get_frame_info, create_video, get_unique_frame_ids
import cv2
from tqdm import tqdm


json_files = os.listdir("data/gamestate_output")


for json_file in json_files:
    json_file = f"data/gamestate_output/{json_file}"
    if "action" not in json_file:
        continue

    # Check if file exists
    if not os.path.exists(json_file):
        msg = f"Error: JSON file '{json_file}' not found."
        raise ValueError(msg)

    # Get filename without extension for output folder
    filename = Path(json_file).stem

    # Create output directories
    output_frames_dir = Path(f"visualization/output/{filename}/frames")
    output_video_dir = Path(f"visualization/output/{filename}/video")

    # Check if output directories already exist and contain files
    if (output_frames_dir.exists() and any(output_frames_dir.iterdir())) or \
        (output_video_dir.exists() and any(output_video_dir.iterdir())):
        response = input(f"Output files already exist for {filename}. Overwrite? (y/n): ")
        if response.lower() == 'y':
            # Remove existing files
            if output_frames_dir.exists():
                shutil.rmtree(output_frames_dir)
            if output_video_dir.exists():
                shutil.rmtree(output_video_dir)
            print("Existing files removed.")
        else:
            msg = "Operation cancelled."
            raise ValueError(msg)

    # Create directories
    output_frames_dir.mkdir(parents=True, exist_ok=True)
    output_video_dir.mkdir(parents=True, exist_ok=True)

    # Get unique frame IDs
    frame_ids = get_unique_frame_ids(json_file)

    if not frame_ids:
        msg = "Error: No frames found in the JSON file"
        raise ValueError(msg)

    print(f"Processing {len(frame_ids)} frames ...")

    # Process frames sequentially
    frame_paths = []

    # Function to process a single frame
    def process_frame(frame_id: str) -> Optional[Path]:
        output_path = output_frames_dir / f"frame_{frame_id}.png"
        
        # Skip if file already exists
        if output_path.exists():
            return output_path
        
        # Get frame info and draw the frame
        frame_info = get_frame_info(json_file, frame_id)
        image = draw_frame(frame_info)
        
        # Save the frame
        cv2.imwrite(str(output_path), image)
        
        return output_path

    # Process frames sequentially with a progress bar
    for frame_id in tqdm(frame_ids, desc="Processing frames"):
        frame_path = process_frame(frame_id)
        if frame_path:
            frame_paths.append(frame_path)

    print(f"Completed! All frames saved to {output_frames_dir}")

    # Create video
    video_output_path = output_video_dir / f"{filename}.mp4"
    create_video(frame_paths, video_output_path, fps=10)

    print(f"Visualization completed for {json_file}")
    print(f"Frames saved to {output_frames_dir}")
    print(f"Video saved to {video_output_path}")

Existing files removed.
Processing 126 frames ...


Processing frames: 100%|██████████| 126/126 [00:03<00:00, 38.96it/s]


Completed! All frames saved to visualization/output/action_1037/frames
Creating video from 126 frames...


Creating video: 100%|██████████| 126/126 [00:01<00:00, 82.44it/s]


Video created successfully: visualization/output/action_1037/video/action_1037.mp4
Visualization completed for data/gamestate_output/action_1037.json
Frames saved to visualization/output/action_1037/frames
Video saved to visualization/output/action_1037/video/action_1037.mp4
Existing files removed.
Processing 126 frames ...


...

Video created successfully: visualization/output/action_1013/video/action_1013.mp4
Visualization completed for data/gamestate_output/action_1013.json
Frames saved to visualization/output/action_1013/frames
Video saved to visualization/output/action_1013/video/action_1013.mp4
Existing files removed.
Processing 126 frames ...


Processing frames: 100%|██████████| 126/126 [00:03<00:00, 40.71it/s]


Completed! All frames saved to visualization/output/action_1009/frames
Creating video from 126 frames...


Creating video: 100%|██████████| 126/126 [00:01<00:00, 75.71it/s]

Video created successfully: visualization/output/action_1009/video/action_1009.mp4
Visualization completed for data/gamestate_output/action_1009.json
Frames saved to visualization/output/action_1009/frames
Video saved to visualization/output/action_1009/video/action_1009.mp4
